In case BooleanNet doesn't run do the following:

* Go to the Files list and open a terminal (`terminal.term`)
* Type: `pip2 install BooleanNet`
* Go to Settings and click on `Restart project`



In [7]:
import boolean2 as b2
import matplotlib.pyplot as plt

### Exercise 1
* Create a Boolean model with three nodes (A, B, C) and the following update rules:  
*A activates B*  
*B activates C*  
*C activates A*  
* Assuming that all nodes are initially active, simulate 10 time steps and print the results. Does the model converge to a steady state or a limit cycle?

In [9]:
model_definition = """

A = B = C = True

A* = C
B* = A
C* = B

"""

* Now change the third rule to *C inhibits A*, simulate 10 time steps and print the results. Does the model converge to a steady state or a limit cycle?

In [3]:
import boolean2

model_definition = """

A = B = C = True

A* = not C
B* = A
C* = B

"""

model = boolean2.Model(text=model_definition, mode='sync')
model.initialize()
model.iterate(steps=10)

for node in model.data:
    print node, model.data[node]
    
model.report_cycles()

A [True, False, False, False, True, True, True, False, False, False, True]
C [True, True, True, False, False, False, True, True, True, False, False]
B [True, True, False, False, False, True, True, True, False, False, False]
No cycle or steady state could be detected from the 11 states


### Exercise 2
* Build a model of the EGF system shown in the lecture slides.
* Determine the steady states or limit cycles for EGF set *ON* or *OFF*, respectively.

In [11]:
import boolean2

EGF_On_model_definition = """

EGF  = True
Sos1 = Rac1 = Erk1 = Pak1 = False

Sos1* = EGF or not Erk1
Rac1* = Sos1
Erk1* = Rac1
Pak1* = Sos1 and Rac1

"""
print "When EGF is On:"

model = boolean2.Model(text=EGF_On_model_definition, mode='sync')
model.initialize()
model.iterate(steps = 10)

for node in model.data:
    print node, model.data[node]
    
model.report_cycles()

When EGF is On:
Erk1 [False, False, False, True, True, True, True, True, True, True, True]
Rac1 [False, False, True, True, True, True, True, True, True, True, True]
EGF [True, True, True, True, True, True, True, True, True, True, True]
Sos1 [False, True, True, True, True, True, True, True, True, True, True]
Pak1 [False, False, False, True, True, True, True, True, True, True, True]
Steady state starting at index 3 -> State: EGF=True, Erk1=True, Pak1=True, Rac1=True, Sos1=True


In [12]:
print "When EGF is Off"

EGF_Off_model_definition = """

 
EGF = Sos1 = Rac1 = Erk1 = Pak1 = False

Sos1* = EGF or not Erk1
Rac1* = Sos1
Erk1* = Rac1
Pak1* = Sos1 and Rac1

"""

model = boolean2.Model(text=EGF_Off_model_definition, mode='sync')
model.initialize()
model.iterate(steps = 10)

for node in model.data:
    print node, model.data[node]

    
model.report_cycles()

When EGF is Off
Erk1 [False, False, False, True, True, True, False, False, False, True, True]
Rac1 [False, False, True, True, True, False, False, False, True, True, True]
EGF [False, False, False, False, False, False, False, False, False, False, False]
Sos1 [False, True, True, True, False, False, False, True, True, True, False]
Pak1 [False, False, False, True, True, False, False, False, False, True, True]
No cycle or steady state could be detected from the 11 states


- Using the matplotlib `imshow` function \(see [https://matplotlib.org/stable/api/\_as\_gen/matplotlib.pyplot.imshow.html](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.imshow.html)\), plot the results as a colour\-coded map where each row contains a protein and each column a time step.



In [13]:
import numpy as np

graph_list = []

for node in model.data:
    graph_list.append(model.data[node])
    
graph_array = np.array(graph_list)    

plt.figure(figsize=(7,7))
plt.yticks(range(0,5),model.data)
plt.imshow(graph_array)



* Run the same model 10 times using an asynchronous update mode and plot the average state over time for each protein, using the same `imshow` function.  
* Compare with the previous plot and discuss the results.

In [23]:
model = boolean2.Model(text=EGF_Off_model_definition, mode='async')
model.initialize()
model.iterate(steps = 10)

import numpy as np

graph_async_list = []

for node in model.data:
    graph_async_list.append(model.data[node])
    
graph_async_array = np.array(graph_async_list)    

plt.figure(figsize=(7,7))
plt.yticks(range(0,5),model.data)
plt.imshow(graph_async_array)


In [18]:
from boolean2 import util

coll = util.Collector()

for i in range(10):
    model = boolean2.Model(text=EGF_Off_model_definition, mode='async')
    model.initialize()
    model.iterate( steps=10 )
    
    # takes all nodes
    nodes = model.nodes
    coll.collect( states=model.states, nodes=nodes )
    
avgs = coll.get_averages( normalize=True )
averages_array = np.array(avgs)

plt.figure(figsize=(7,7))
plt.yticks(range(0,5),model.data)
plt.imshow(averages_array)

TypeError: Image data cannot be converted to float